# 4章　制御対象の振る舞い

ライブラリの読み込みと関数の定義

In [ ]:
%pip install sympy

In [ ]:
%pip install control

In [ ]:
%pip install numpy scipy control

In [1]:
# 必要なライブラリをインポート
import matplotlib.pyplot as plt  # グラフ描画用ライブラリmatplotlibのpyplotモジュールをインポート
import numpy as np               # 数値計算用ライブラリNumPyをインポート　数値計算において、特に行列の計算や多次元配列の操作を効率的に行いたい場合に使用する。

# グラフ全体のデフォルト設定を変更

plt.rcParams['font.family'] = 'sans-serif'    # 使用するフォントをサンセリフ体に設定
plt.rcParams['xtick.direction'] = 'in'         # x軸の目盛線を内向きに設定
plt.rcParams['ytick.direction'] = 'in'         # y軸の目盛線を内向きに設定
plt.rcParams['xtick.major.width'] = 1.0         # x軸の主目盛線の太さを1.0に設定
plt.rcParams['ytick.major.width'] = 1.0         # y軸の主目盛線の太さを1.0に設定
plt.rcParams['font.size'] = 10                  # フォントサイズを10に設定
plt.rcParams['axes.linewidth'] = 1.0             # 囲み線（軸枠）の太さを1.0に設定
plt.rcParams['mathtext.default'] = 'regular'     # 数式フォントを通常の文字フォントに統一
plt.rcParams['axes.xmargin'] = '0'               # x軸方向の余白を0に設定（データ端までプロット）
plt.rcParams['axes.ymargin'] = '0.05'             # y軸方向の余白を5%に設定
plt.rcParams['savefig.facecolor'] = 'None'        # グラフ保存時の背景色を透明に設定
plt.rcParams['savefig.edgecolor'] = 'None'        # グラフ保存時の枠線色を透明に設定

In [2]:
# 線種を変更するジェネレータ
def linestyle_generator():
    linestyle = ['-', '--', '-.', ':']  # 使用する線種のリスト（実線、破線、一点鎖線、点線）
    lineID = 0                          # 最初の線種のインデックス
    while True:
        yield linestyle[lineID]         # 現在の線種を返す
        lineID = (lineID + 1) % len(linestyle)  # 線種を順番に切り替え（ループする）

# 時間応答のグラフを整える関数
def plot_set(fig_ax, *args):
    fig_ax.set_xlabel(args[0])           # x軸ラベルを設定
    fig_ax.set_ylabel(args[1])           # y軸ラベルを設定
    fig_ax.grid(ls=':')                  # 格子線を点線（:）で表示
    if len(args) == 3:
        fig_ax.legend(loc=args[2])       # 第3引数があれば、凡例を指定した位置に表示

# ボード線図（周波数応答グラフ）を整える関数
def bodeplot_set(fig_ax, *args):
    fig_ax[0].grid(which='both', ls=':')    # ゲインプロットに格子線を表示
    fig_ax[0].set_ylabel('Gain [dB]')       # ゲインプロットのy軸ラベルを設定
    
    fig_ax[1].grid(which='both', ls=':')    # 位相プロットに格子線を表示
    fig_ax[1].set_xlabel('$\\omega$ [rad/s]')  # 位相プロットのx軸ラベル（角周波数ω）
    fig_ax[1].set_ylabel('Phase [deg]')     # 位相プロットのy軸ラベル

    if len(args) > 0:
        fig_ax[1].legend(loc=args[0])       # 第1引数があれば、位相プロットに凡例を設定
    if len(args) > 1:
        fig_ax[0].legend(loc=args[1])       # 第2引数があれば、ゲインプロットに凡例を設定


# 4.1　時間応答 p.94

ここでは、1次遅れ系と2次遅れ系を対象としている。

そして、ステップ上の入力(ステップ入力)

<img src="png/4_9.png" alt="1" width="300" height="150">

を加えたときの出力の振る舞いである*ステップ応答*をみていく

Pythonでは、step関数を使ってステップ応答を計算する

「 y, t = step(sys, Td) 」

引数は sys と Td で、sys は伝達関数モデルまたは状態空間モデル、 Td はシミュレーション時間。Td は、np.arange(0, 5, 0.01)のようにして与えるが、省略することもできる。

step の返り値は y と t で、y がモデルの出力応答、t が時間

In [3]:
# control.matlab モジュールから
# ・tf: 伝達関数（Transfer Function）を作成する関数
# ・step: ステップ応答（Step Response）を計算・描画する関数
from control.matlab import tf, step

## 4.1.1　1次遅れ系

入力が変化したときに、すぐに出力が追いつかず、ゆっくり時間をかけて変化するようなシステムのことを指す。

K：ゲイン（最終的な出力の倍率） T：時定数（応答の速さを決める） s：ラプラス変換での変数

<img src="png/4_1.png" alt="1" width="400" height="250">

この形で表現できるシステムを1次遅れ系と呼ぶ 。

そして、1次遅れ系のKをゲイン、Tを時定数という。とくに時定数は、応答の速さ(速応性)を決める重要なパラメータ。

### 図1　1次遅れ系のステップ応答

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

時定数T = 0.5、ゲインK = 1を設定してください。

In [ ]:
# 交点に補助線（水平線・垂直線）とマーカーを描画する関数
def cross_lines(x, y, **kwargs):
    ax = plt.gca()                        # 現在の座標軸を取得
    ax.axhline(y, **kwargs)                # 水平線を引く（y座標位置）
    ax.axvline(x, **kwargs)                # 垂直線を引く（x座標位置）
    ax.scatter(T, 0.632, **kwargs)          # 交点にマーカーを描画


# ---------------------------------------------------------------
# システムのパラメータ設定
T, K =                               # 時定数T=0.5、ゲインK=1を設定
# ---------------------------------------------------------------


# 1次遅れ系の伝達関数を定義
P = tf([0, K], [T, 1])                     # P(s) = K / (Ts + 1)
# ステップ応答を計算
y, t = step(P, np.arange(0, 5, 0.01))       # 時間0〜5秒、刻み幅0.01秒で計算
# グラフを作成
fig, ax = plt.subplots(figsize=(3, 2.3))    # 図と座標軸を作成（サイズ指定）
# ステップ応答をプロット
ax.plot(t, y)


# 時定数の位置に補助線とマーカーを描画
cross_lines(T, 0.632, color='k', lw=0.5)     # 黒色、線幅0.5
# 交点の座標を注釈として表示
ax.annotate('$(0.5, 0.632)$', xy=(0.7, 0.5)) # 位置(0.7, 0.5)に注釈を配置
# x軸の目盛りを設定
ax.set_xticks(np.linspace(0, 5, 6))          # 0から5までを6等分した目盛り
# グラフのラベル・グリッドを整える
plot_set(ax, 't', 'y')                       # x軸ラベルを't'、y軸ラベルを'y'に設定

時定数と応答の関係　1次遅れ系のステップ応答(Tを変化させる)

時定数 T を変化させたとき、応答がどのように変わるかを確認する。定常値の63.2% に到達する時間が時定数なので、T を小さくすると応答が速くなり、逆に大きくすると応答が遅くなることが予想できる。

### 図2　時定数 T を T = 1, 0.5, 0.1 と変化させた時のステップ応答

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

ゲインK = 1を設定してください。

また、時定数 T を T = 1, 0.5, 0.1 と変化させてください

In [ ]:
# 線種ジェネレータを初期化（プロットごとに異なる線種を使うため）
LS = linestyle_generator()

# グラフの準備（fig: 図全体、ax: 座標軸オブジェクト、サイズ指定）
fig, ax = plt.subplots(figsize=(3, 2.3))


# ---------------------------------------------------------------
# システムのパラメータ設定
K =                 # ゲイン K を 1 に設定
T = []              # 時定数 T のリスト（大→小：応答が遅い→速い）
# ---------------------------------------------------------------


# 各 T に対して、1次遅れ系のステップ応答を計算してプロット
for i in range(len(T)):
    # 伝達関数 P(s) = K / (T[i]s + 1) のステップ応答を計算（0〜5秒の範囲）
    y, t = step(tf([0, K], [T[i], 1]), np.arange(0, 5, 0.01))
    # 線種を変えてステップ応答を描画、凡例に T の値を表示
    ax.plot(t, y, ls=next(LS), label=f'T={T[i]}')

# x軸の目盛りを0〜5の範囲で6等分、y軸を0〜1で6等分に設定
ax.set_xticks(np.linspace(0, 5, 6))
ax.set_yticks(np.linspace(0, 1, 6))

# 軸ラベル、グリッド、凡例（'best'な位置）などを設定する関数を呼び出し
plot_set(ax, 't', 'y', 'best')

「時定数 T が負の値の場合」のステップ応答をプロットし、1次遅れ系の安定性について理解するための例

Tが負の値のときは一定値に収束せず、発散してしまう

### 図3　 T = -1 の場合のステップ応答

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

時定数T = -1、ゲインK = 1を設定してください。

In [ ]:
# 描画用の図と軸オブジェクトを生成（サイズは3インチ×2.3インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))


# ---------------------------------------------------------------
# システムのパラメータ設定
# ※ Tが負の値の場合、システムは不安定となる
T, K =                   # 時定数Tを-1（負の値）、ゲインKを1に設定
# ---------------------------------------------------------------


# 1次遅れ系の伝達関数 P(s) = K / (T*s + 1) を定義
# ここでは T = -1 のため、極が右半平面にあり不安定
# ステップ応答を 0秒〜5秒の範囲、0.01秒刻みで計算
y, t = step(tf([0, K], [T, 1]), np.arange(0, 5, 0.01))

# ステップ応答を黒色でプロット
ax.plot(t, y, color='k')

# x軸の目盛りを1秒間隔で設定（0〜5秒）
ax.set_xticks(np.arange(0, 5.2, step=1.0))

# 軸ラベルとグリッドを設定（x: t, y: y）
plot_set(ax, 't', 'y')

ゲインと応答の関係　1次遅れ系のステップ応答(Kを変化させる)
        
K を大きくすると、y の定常値が変わる。そして、定常値が y(∞) = K， つまりゲイン K の値であることもわかる。

### 図4　ゲインKを K = 1, 2, 3 と変化させたときのステップ応答

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

時定数T = 0.5を設定してください

また、ゲインKを K = 1, 2, 3 と変化させてください

In [ ]:
# 線種ジェネレータを初期化（'–', '--', '-.', ':' の順にループ）
LS = linestyle_generator()
# 描画用の図と軸オブジェクトを生成（サイズは3インチ×2.3インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))


# ---------------------------------------------------------------
# システムのパラメータ設定
T =                     # 時定数 T を 0.5に設定
K = []                  # ゲインKの異なる3つの値を設定
# ---------------------------------------------------------------


# 各Kに対して、ステップ応答を計算してプロット
for i in range(len(K)):
    # 伝達関数 P(s) = K / (Ts + 1) を作成し、ステップ応答を求める
    y, t = step(tf([0, K[i]], [T, 1]), np.arange(0, 5, 0.01))
    # ステップ応答を異なる線種でプロット（ラベル付き）
    ax.plot(t, y, ls=next(LS), label=f'K={K[i]}')

# x軸の目盛りを0〜5秒まで1秒刻みで設定
ax.set_xticks(np.arange(0, 5.2, step=1.0))
# y軸の目盛りを0〜3まで1刻みで設定
ax.set_yticks(np.arange(0, 3.2, step=1))

# 軸ラベルとグリッド、凡例（左上）を設定
plot_set(ax, 't', 'y', 'upper left')

以上の結果を、台車の場合に対応付けて考えてみる。

まず、車輪にモータが付いている台車を考える。このとき、モータに一定の電圧を印加すると、台車は動き出し、やがて一定の速度（等速直線運動）で動き続けるようになる。

これを表したものが、図1。

台車の質量が小さいほど（＝慣性が小さいほど）、より速く一定の速度に達する。これは、「$T = \frac{M}{\mu}$」の値が小さくなることに対応している。

また、粘性（摩擦）が低いほど、一定の速度に落ち着くまでの時間が長くなり、さらに、最終的な速度も大きくなる（図2）。

これは、「$T = \frac{M}{\mu}$」が大きくなること、「$K = \frac{1}{\mu}$」が大きくなることに対応している。

## 1次遅れ系の時間応答の計算

制御対象 $P(s)$ の出力 $y(s)$ は $y(s) = P(s)u(s)$ で表される。

ここで，ステップ入力の場合，$u(s) = \frac{1}{s}$ となるので，出力は，$y(s) = \frac{P(s)}{s}$ となる。

したがって，時間応答 $y(t)$ を求めるには，$y(s)$ を逆ラプラス変換すればよいので，$$y(t) = \mathcal{L}^{-1}[y(s)] = \mathcal{L}^{-1}\left[\frac{P(s)}{s}\right] \quad (1)$$を計算すればよいことになる。

なお，逆ラプラス変換は，$$f(t) = \mathcal{L}^{-1}[f(s)] = \frac{1}{2\pi j} \int_{c-j\infty}^{c+j\infty} f(s)e^{st} ds \quad (t > 0) \quad (2)$$ですが，この計算を行う必要はない。

すでにわかっている基本的な関数のラプラス変換を思い出しながら（275 ページのラプラス変換表を参照しながら），逆変換をする。

1次遅れ系の場合には，$$y(s) = \frac{K}{1+Ts} \frac{1}{s} = K\left(\frac{1}{s} - \frac{T}{1+Ts}\right) = K\left(\frac{1}{s} - \frac{1}{s+\frac{1}{T}}\right) \quad (3)$$となるので，これを逆ラプラス変換して，$$y(t) = K\left(1 - e^{-\frac{t}{T}}\right) \quad (4)$$が得られる。

ここで，$y(0)=0$ で，$T>0$ のとき $y(\infty) = K$ 。そして，$y(T) = 1 - e^{-1} = 0.632$ であることから，これをプロットしたものが，図1 であることがわかる。

なお，ラプラス変換の性質の1つとして，$$y(\infty) = \lim_{t \to \infty} y(t) = \lim_{s \to 0} sy(s) \quad (5)$$という関係（最終値の定理）がある。これを用いると，$$y(\infty) = \lim_{s \to 0} sP(s)u(s) = \lim_{s \to 0} sP(s)\frac{1}{s} = P(0) = K \quad (6)$$が得られる。

式(3)における部分分数分解は，Python では，つぎのように数式処理によって求まる。

In [ ]:
import sympy as sp  # 数式処理ライブラリ sympy をインポート　 数式の簡略化、展開、微積分、ラプラス変換など、数式の操作をシンボリックに実行することができる。
# sp.init_printing()  # 数式の表示を見やすく整える（Jupyter等で有効）

s = sp.Symbol('s')  # ラプラス変換の複素数変数 s を定義
T = sp.Symbol('T', real=True)  # 時定数 T を実数として定義

# ラプラス変換された伝達関数 P(s) = 1 / [(1 + T*s) * s]
# これは積分器と1次遅れ系を直列接続した系に相当
P = 1 / ((1 + T*s) * s)

# P(s) を変数 s について部分分数分解する
# これにより、ラプラス逆変換がしやすい形に変換できる
sp.apart(P, s)

<img src="png/4_3.png" alt="1" width="400" height="250">

逆ラプラス変換は、SymPy を用いて求めることができる。

In [ ]:
import sympy as sp  # 数式処理ライブラリ sympy をインポート　 数式の簡略化、展開、微積分、ラプラス変換など、数式の操作をシンボリックに実行することができる。
# sp.init_printing()  # 数式の出力を見やすくする（Jupyter Notebookなどで効果的）

s = sp.Symbol('s')  # ラプラス変換に用いる複素数変数 s を定義
t = sp.Symbol('t', positive=True)  # 時間変数 t を正の実数として定義
T = sp.Symbol('T', real=True)  # 時定数 T を実数として定義

# 以下の式のラプラス逆変換を求める：
# L^{-1} [1/s - 1/(s + 1/T)] 
# これは、単位ステップ関数（1/s）の応答と、指数関数（1/(s + a)）の応答の差に相当
sp.inverse_laplace_transform(1/s - 1/(s + 1/T), s, t)

## 4.1.2　2次遅れ系　(p.100)

RLC 回路の伝達関数モデル (76 ページ) は，$$P(s) = \frac{1}{CLs^2 + CRs + 1} = \frac{\frac{1}{CL}}{s^2 + \frac{R}{L}s + \frac{1}{CL}} \quad (4.10)$$なので，$$K=1, \quad \omega_n = \sqrt{\frac{1}{CL}}, \quad \zeta = \frac{R}{2}\sqrt{\frac{C}{L}} = \frac{R}{2L\omega_n} \quad (4.11)$$とすると，$$P(s) = \frac{K\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2} \quad (4.12)$$で表現できる (アームの伝達関数モデル (76 ページ) も同じ形で表すことができる)。

このように表現できるシステムを 2次遅れ系と呼ぶ。そして，$\zeta$ を減衰係数，$\omega_n$ を固有角周波数と呼ぶ。

それでは，この2次遅れ系のステップ応答をみてみよう (以下では基本的に，$K=1$ とする)。

リスト 4.7 を実行すると，図 4.6 が得られます。初期値 0 からスタートして，徐々に値が大きくなり，$T_p = 0.685$ 秒あたりで最大値 $y_{max} = 1.25$ をとり，その後，1 に収束していく。

なお，最大値と最終的な値の差 (この場合，$1.25 - 1 = 0.25$) のことを最大行き過ぎ量 (オーバーシュート) という。

1 次遅れ系の場合は，オーバーシュートは生じないが、2次遅れ系の場合は、生じることがある。

<img src="png/4_5.png" alt="1" width="500" height="350">

In [6]:
# 制御工学用ライブラリ control.matlab モジュールから
# 伝達関数を定義する関数 tf と、ステップ応答を計算する関数 step をインポート
from control.matlab import tf, step

In [7]:
# クロスラインを描画する関数（水平線・垂直線・点を描画）
def cross_lines(x, y, **kwargs):
    plt.gca()  # 現在のAxesを取得（この行は無くても動くが、ここでは取得のみで使っていない）
    ax.axhline(y, **kwargs)  # y座標に水平線を引く
    ax.axvline(x, **kwargs)  # x座標に垂直線を引く
    ax.scatter(x, y, **kwargs)  # 交点に点を描画する

### 図4.6　　2次遅れ系のステップ応答  p.101

In [ ]:
# 減衰係数 zeta と 固有角周波数 omega_n を設定
(zeta, omega_n) = (0.4, 5)

# 2次遅れ系の伝達関数 P(s) を作成
P = tf([0, omega_n**2], [1, 2*zeta*omega_n, omega_n**2])

# 伝達関数Pのステップ応答を計算
y, t = step(P, np.arange(0, 5, 0.01))

# グラフ描画の準備
fig, ax = plt.subplots(figsize=(3, 2.3))
ax.plot(t, y)  # ステップ応答をプロット

# 最大行き過ぎ量（オーバーシュート）とピーク時間を計算
ymax = 1 + 1 * np.exp(-(np.pi*zeta)/np.sqrt(1-zeta**2))  # オーバーシュート値
Tp = np.pi/omega_n/np.sqrt(1-zeta**2)  # ピーク時間

# ピーク位置にクロスラインと点を描画
cross_lines(Tp, ymax, color='k', lw=0.5)

# ピーク位置に注釈を付加
ax.annotate('$(T_P, y_{max})$', xy=(1.2, 1.1))

# ピーク値とピーク時間をコンソールに表示
print('ymax=', ymax)
print('Tp=', Tp)

# x軸・y軸の目盛り設定
ax.set_xticks(np.arange(0, 5.2, step=1.0))
ax.set_yticks(np.arange(0, 1.3, step=0.25))

# 軸ラベルなどを設定する（plot_set関数を使用）
plot_set(ax, 't', 'y')

初期値0からスタートして、徐々に値が大きくなり、 Tp = 0.685 秒あたりで最大値 ymax = 1.25 をとり、その後、1に収束している。

#### 最大値と最終的な値の差　(この場合、1.25 - 1 = 0.25)　のことを 最大行き過ぎ量(オーバーシュート) という。

1次遅れ系の場合は、 オーバーシュートは生じなかったが、2次遅れ系の場合は、生じることがある。

### 図4.7　　「減衰係数 ${\zeta}$ を  ${\zeta}$ = 1, 0.7, 0.4 と変化させたときのステップ応答」　2次遅れ系のステップ応答(ζを変化させる) p.102

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

減衰係数zetaのリスト（1、0.7、0.4）に設定

固有角周波数omega_nを5に設定

In [ ]:
# さまざまな線種（linestyle）を生成するジェネレータを用意
LS = linestyle_generator()
# グラフ描画の準備
fig, ax = plt.subplots(figsize=(3, 2.3))


# ---------------------------------------------------------------
# システムのパラメータ設定
# 減衰係数zetaのリスト（1、0.7、0.4）を設定
zeta = []
# 固有角周波数omega_nを5に設定
omega_n = 
# ---------------------------------------------------------------


# 減衰係数ごとにループして処理
for i in range(len(zeta)):
    # それぞれの減衰係数に対して2次遅れ系の伝達関数P(s)を作成
    P = tf([0, omega_n**2], [1, 2*zeta[i]*omega_n, omega_n**2])
    # ステップ応答を計算
    y, t = step(P, np.arange(0, 5, 0.01))
    # プロット用の引数を作成（線種とラベル設定）
    pltargs = {'ls': next(LS), 'label': f'$\\zeta$={zeta[i]}'}
    # 応答をプロット
    ax.plot(t, y, **pltargs)

# x軸の目盛り設定（0〜5を1刻み）
ax.set_xticks(np.arange(0, 5.2, step=1.0))
# y軸の目盛り設定（0〜1.25を0.25刻み）
ax.set_yticks(np.arange(0, 1.3, step=0.25))

# 軸ラベル、タイトル、凡例などを整える（plot_set関数使用）
plot_set(ax, 't', 'y', 'best')

${\zeta}$ を ${\zeta}$ = 1, 0.7,0.4 としたときの結果が 図4.7。

これより、 ${\zeta}$ = 1 のときはオーバーシュートが生じていないが、 ${\zeta}$ = 0.7, 0.4 のときは生じていることがわかる。また、オーバーシュートの大きさは、${\zeta}$ の値が小さくなるほど大きくなることが確認できる。

### 図4.8 　減衰係数を　${\zeta}$　=　0.1,　0，ー0.05　と変化させたときのステップ応答 p.102

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

減衰係数zetaのリスト（0.1,　0，ー0.05）に設定

固有角周波数omega_nを5に設定

In [ ]:
# ラインスタイルのジェネレータを初期化
LS = linestyle_generator()

# 図と座標軸を作成（サイズは3×2.3インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))


# ---------------------------------------------------------------
# システムのパラメータ設定
# 減衰係数のリスト（0.1：軽減衰、0：無減衰、-0.05：発散系）
zeta = []
# 固有角周波数を設定
omega_n = 
# ---------------------------------------------------------------


# 各減衰係数に対してループ
for i in range(len(zeta)):
    # 2次遅れ系の伝達関数 P(s) = omega_n^2 / (s^2 + 2*zeta*omega_n*s + omega_n^2)
    P = tf([0, omega_n**2], [1, 2*zeta[i]*omega_n, omega_n**2])
    # ステップ応答を計算（0秒〜5秒を0.01刻みで）
    y, t = step(P, np.arange(0, 5, 0.01))
    # ラベルと線種を指定してプロット
    pltargs = {'ls': next(LS), 'label': f'$\\zeta$={zeta[i]}'}
    ax.plot(t, y, **pltargs)

# x軸の目盛りを1刻みに設定
ax.set_xticks(np.arange(0, 5.2, step=1.0))
# y軸の目盛りを-4から4まで2刻みに設定（発散も視野に入れるため）
ax.set_yticks(np.arange(-4, 5, step=2))
# 軸ラベルと凡例の設定（凡例の位置：左下）
plot_set(ax, 't', 'y', 'lower left')

さらに、${\zeta}$ を ${\zeta}$ = 0.1, 0, ー0.05 としたときの結果が図4.8。

この図から、 ${\zeta}$ = 0.1 のときは振動しながら 1.0 に収束しているが、 ${\zeta}$ = 0 のときは振動が続いて一定値に収束しないことがわかる。そして、${\zeta}$ = ー0.05 にすると、振動が徐々に大きくなり、発散していくことがわかる。

このように、減衰係数は減衰性(安定度)を決めるパラメータで、${\zeta}$ が正の値のとき、出力が一定値に収束し、負の値のときに発散する。そして、0 < ${\zeta}$ < 1 のときは振動しながら収束し、${\zeta}$ ≥ 1のときは振動せずに収束する。

### 図4.9　固有角周波数 ${\omega_n}$ を ${\omega_n}$ = 1, 5 ,10 と変化させたときのステップ応答 p.103

以下のパラメータをプログラムに実装し、波形を表示させてください。

### システムのパラメータ設定

減衰係数zetaを 0.7 に設定

固有角周波数omega_nのリスト（1, 5 ,10）に設定

In [ ]:
# ラインスタイルを順番に切り替えるジェネレータを初期化
LS = linestyle_generator()

# 図と座標軸を作成（サイズは3×2.3インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))

# ---------------------------------------------------------------
# システムのパラメータ設定
# 減衰係数 zeta を設定（今回はすべて 0.7 に固定）
zeta = 
# 固有角周波数 omega_n のリスト（1, 5, 10 を設定）
omega_n = []
# ---------------------------------------------------------------



# 各 omega_n に対してループ処理
for i in range(len(omega_n)):
    # 2次遅れ系の伝達関数を定義
    # P(s) = omega_n^2 / (s^2 + 2*zeta*omega_n*s + omega_n^2)
    P = tf([0, omega_n[i]**2], [1, 2*zeta*omega_n[i], omega_n[i]**2])
    # ステップ応答を計算（0秒〜5秒を0.01秒刻みで）
    y, t = step(P, np.arange(0, 5, 0.01))
    # 線種とラベルを指定して応答をプロット
    pltargs = {'ls': next(LS), 'label': f'$\\omega_n$={omega_n[i]}'}
    ax.plot(t, y, **pltargs)

# x軸の目盛りを1秒刻みに設定
ax.set_xticks(np.arange(0, 5.2, step=1.0))

# 軸ラベル、グリッド、凡例（最適な位置）を設定
plot_set(ax, 't', 'y', 'best')

これより、${\omega_n}$ を大きくすると、応答が速くなることがわかる。オーバーシュートの大きさは変わらない。 つまり、固有角周波数 ${\omega_n}$ は、1次遅れ系のときの時定数 $T$ と同じように、速応性を決めるパラメータであることがわかる。

## 2次遅れ系の時間応答の計算　p.104

基本式2次遅れ系の伝達関数 $G(s)$ は以下の式で表される。

### $G(s) = \frac{K\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2}$

ここへ、入力としてステップ入力 $U(s) = \frac{1}{s}$ を加えたときの出力 $Y(s)$ は次のようになる。

### $Y(s) = G(s)U(s) = \frac{K\omega_n^2}{s(s^2 + 2\zeta\omega_n s + \omega_n^2)}$

### 1. 臨界減衰（$\zeta = 1$）の場合


$\zeta = 1$ のとき、分母は完全平方式 $(s+\omega_n)^2$ となる。

### $Y(s) = \frac{K\omega_n^2}{s(s+\omega_n)^2} \quad \cdots (4.13)$

部分分数分解この式を逆ラプラス変換するために、以下の形に部分分数分解する。

### $\frac{K\omega_n^2}{s(s+\omega_n)^2} = K \left( \frac{A}{s} + \frac{B}{s+\omega_n} + \frac{C}{(s+\omega_n)^2} \right)$

係数 $A, B, C$ を求める。

#### $A$ の導出: 両辺に $s$ を掛けて $s=0$ を代入

### $A = \left. \frac{\omega_n^2}{(s+\omega_n)^2} \right|_{s=0} = \frac{\omega_n^2}{\omega_n^2} = 1$

#### $C$ の導出: 両辺に $(s+\omega_n)^2$ を掛けて $s=-\omega_n$ を代入

### $C = \left. \frac{\omega_n^2}{s} \right|_{s=-\omega_n} = -\omega_n$

#### $B$ の導出: 通分して分子の係数を比較、あるいは $s$ に適当な値（無限大の極限など）を代入して求めると、$B = -1$ となる。

これらを代入すると、式(4.14)が得られる。

### $Y(s) = K \left( \frac{1}{s} - \frac{1}{s+\omega_n} - \frac{\omega_n}{(s+\omega_n)^2} \right) \quad \cdots (4.14)$

#### 逆ラプラス変換

#### ラプラス変換については前述（1次遅れ系の時間応答の計算）を参照してください

よって、時間応答 $y(t)$ は以下のようになる。

### $y(t) = K \left( 1 - e^{-\omega_n t} - \omega_n t e^{-\omega_n t} \right) \quad \cdots (4.15)$

この式より、$\zeta=1$ の場合は振動（オーバーシュート）せず、$t \to \infty$ で $K$ に収束することがわかる。

In [9]:
# Sympy ライブラリをインポート　 数式の簡略化、展開、微積分、ラプラス変換など、数式の操作をシンボリックに実行することができる。
import sympy as sp

# 数式をきれいに表示できるように設定
# sp.init_printing()

# 複素変数 s を定義
s = sp.Symbol('s')
# 実数型の記号 T を定義（システムの時定数などに使う）
T = sp.Symbol('T', real=True)
# 伝達関数 P(s) を定義
# P(s) = 1 / [(1 + T*s) * s]
# これは1次遅れ要素と積分要素を持つシステム
# P = 1/((1+T*s)*s)

# 部分分数展開（apart）を行う
# P(s) を簡単な分数の足し算に分解
# sp.apart(P, s)

逆ラプラス変換

In [10]:
# 時間領域の変数 t を定義（正の値のみを想定）
t = sp.Symbol('t', positive=True)

# ラプラス変換の逆変換を行う
# 与えられた式：1/s - 1/(s + 1/T)
# これは P(s) = 1/s - 1/(s + 1/T) の逆ラプラス変換を計算して、時間領域の関数に変換する
# sp.inverse_laplace_transform(1/s - 1/(s + 1/T), s, t)

部分分数展開（２次遅れ系のステップ応答；重根）

In [ ]:
# 実数と仮定した記号 w を定義
w = sp.Symbol('w', real=True)
# 伝達関数 P(s) を定義
# P(s) = (w^2) / (s * (s + w)^2)
P = w**2 / (s * (s + w) ** 2)

# 部分分数分解を行う
# 複雑な分母を持つ式を、簡単な分数の和に分解する
sp.apart(P, s)

逆ラプラス変換

In [ ]:
# 分数式 P(s) を部分分数分解する
# w^2 / (s * (s+w)^2) を、s に関して簡単な分数の和に分解
P = sp.apart(w**2 / s / (s + w)**2, s)

# 部分分数分解された P(s) を逆ラプラス変換し、時間領域の関数 Pt(t) を求める
Pt = sp.inverse_laplace_transform(P, s, t)

# 結果の式を展開して、より見やすい形にする
sp.expand(Pt)

### 2. 振動的および非振動的挙動（$\zeta \neq 1$）の場合

$\zeta \neq 1$ の場合、特性方程式 $s^2 + 2\zeta\omega_n s + \omega_n^2 = 0$ の根を $p_1, p_2$ と置く。解の公式より、根は以下のようになる。

### $p_1, p_2 = -\zeta\omega_n \pm \omega_n\sqrt{\zeta^2 - 1}$

出力 $Y(s)$ は以下のように因数分解できる。

### $Y(s) = \frac{K\omega_n^2}{s(s-p_1)(s-p_2)} \quad \cdots (4.16)$

部分分数分解これを部分分数分解する。

### $\frac{K\omega_n^2}{s(s-p_1)(s-p_2)} = K\omega_n^2 \left( \frac{A}{s} + \frac{B}{s-p_1} + \frac{C}{s-p_2} \right)$

! 係数を求めると：

#### ・$A = \frac{1}{p_1 p_2}$

#### ・$B = \frac{1}{p_1(p_1 - p_2)}$

#### ・$C = \frac{1}{p_2(p_2 - p_1)} = -\frac{1}{p_2(p_1 - p_2)}$

また、解と係数の関係より $p_1 p_2 = \omega_n^2$ なので、全体にかかる係数 $K\omega_n^2$ と相殺などを整理すると、画像にある式(4.17)の形に変形できる。

### $Y(s) = \frac{K\omega_n^2}{p_1 p_2} \left( \frac{1}{s} + \frac{p_2}{(p_1 - p_2)(s - p_1)} - \frac{p_1}{(p_1 - p_2)(s - p_2)} \right)$

ここで $\frac{K\omega_n^2}{p_1 p_2} = K$ なので、逆ラプラス変換を行うと以下のようになる。

### $y(t) = K \left( 1 + \frac{p_2}{p_1 - p_2}e^{p_1 t} - \frac{p_1}{p_1 - p_2}e^{p_2 t} \right) \quad \cdots (4.18)$

#### ！係数の導出方法を知りたい方は、下のセルを開く

### 1. 係数 $A$ の導出

（$s$ の項）分母が $s$ である項の係数 $A$ を求める。両辺に 

#### 1. $s$ を掛ける。

### $\frac{s}{s(s-p_1)(s-p_2)} = A + \frac{sB}{s-p_1} + \frac{sC}{s-p_2}$

左辺の $s$ が約分されて消える。

### $\frac{1}{(s-p_1)(s-p_2)} = A + s(\dots)$

#### 2. $s = 0$ を代入する。右辺の第2項以降はすべて $s$ が掛かっているため $0$ になる。

### $\frac{1}{(0-p_1)(0-p_2)} = A$

#### 3. 計算結果

### $A = \frac{1}{(-p_1)(-p_2)} = \frac{1}{p_1 p_2}$

#### $B = \frac{1}{p_1(p_1 - p_2)}$

#### $C = \frac{1}{p_2(p_2 - p_1)} = -\frac{1}{p_2(p_1 - p_2)}$

### 2. 係数 $B$ の導出（$s-p_1$ の項）
  
分母が $s-p_1$ である項の係数 $B$ を求める。

#### 1. 両辺に $(s-p_1)$ を掛ける。

### $\frac{s-p_1}{s(s-p_1)(s-p_2)} = \frac{A(s-p_1)}{s} + B + \frac{C(s-p_1)}{s-p_2}$

左辺の $(s-p_1)$ が約分されて消える。

### $\frac{1}{s(s-p_2)} = (\dots)(s-p_1) + B + (\dots)(s-p_1)$

#### 2. $s = p_1$ を代入する。右辺の $B$ 以外の項は $(p_1 - p_1) = 0$ となるため消える。

### $\frac{1}{p_1(p_1 - p_2)} = B$

#### 3. 計算結果

### $B = \frac{1}{p_1(p_1 - p_2)}$

### 3. 係数 $C$ の導出（$s-p_2$ の項）

分母が $s-p_2$ である項の係数 $C$ を求める。

#### 1. 両辺に $(s-p_2)$ を掛ける。左辺の $(s-p_2)$ が約分されて消える。

### $\frac{1}{s(s-p_1)} = C + (\text{他の項})$

#### 2. $s = p_2$ を代入。

### $C = \frac{1}{p_2(p_2 - p_1)}$

##### 3. 式の整理（符号の調整）このままでも間違いではないが、係数 $B$ と分母の形（$p_1 - p_2$）を揃えるために、マイナスを前に出して順序を入れ替える。

### $p_2 - p_1 = -(p_1 - p_2)$

これを代入すると、

### $C = \frac{1}{p_2 \{ -(p_1 - p_2) \}} = -\frac{1}{p_2(p_1 - p_2)}$

部分分数展開（２次遅れ系のステップ応答；異なる２つの根）

In [ ]:
# p1, p2 を実数のシンボルとして定義
p1 = sp.Symbol('p1', real=True)
p2 = sp.Symbol('p2', real=True)

# 分数式 P(s) を定義
# P(s) = w^2 / (s*(s-p1)*(s-p2))
P = w**2 / (s * (s - p1) * (s - p2))

# 分数式 P(s) を s に関して部分分数分解する
# → 単純な分数（1次式の積に分解）にして扱いやすくする
sp.apart(P, s)

逆ラプラス変換

In [ ]:
# 分数式 P(s) をラプラス逆変換する
# s領域（周波数領域）の表現 P(s) を、t領域（時間領域）の関数に変換
sp.inverse_laplace_transform(P, s, t)

### 逆ラプラス変換後の一般解（式4.18）に対し、解と係数の関係から得られる以下の関係式を代入する。

* #### $p_1 p_2 = \omega_n^2$

* #### $p_1 - p_2 = 2\omega_n\sqrt{\zeta^2 - 1}$

これらを代入して整理すると、以下の式が得られる。

$\begin{aligned}
y(t) &= K \left( 1 + \frac{-\zeta - \sqrt{\zeta^2-1}}{2\sqrt{\zeta^2-1}} e^{p_1 t} - \frac{-\zeta + \sqrt{\zeta^2-1}}{2\sqrt{\zeta^2-1}} e^{p_2 t} \right) \\[8pt]
&= K \left( 1 - \frac{\zeta + \sqrt{\zeta^2-1}}{2\sqrt{\zeta^2-1}} e^{p_1 t} + \frac{\zeta - \sqrt{\zeta^2-1}}{2\sqrt{\zeta^2-1}} e^{p_2 t} \right) \quad \cdots (4.19)
\end{aligned}$

#### 過減衰（$\zeta > 1$）の場合

このとき、$\sqrt{\zeta^2 - 1}$ は実数となる。したがって、根 $p_1, p_2$ はともに負の実数。

$y(t)$ は $y(0)=0$ から始まり、振動することなく（オーバーシュートせず）指数関数的に目標値 $y(\infty) = K$ へと近づいていく。

--------------------------------------------------------------------------------------------------------------

### 不足減衰（$|\zeta| < 1$）とオイラーの公式

$|\zeta| < 1$ のとき、$\sqrt{\zeta^2 - 1} = j\sqrt{1 - \zeta^2}$ となり、根は複素共役になる。

$p_1, p_2 = -\zeta\omega_n \pm j\omega_n\sqrt{1 - \zeta^2}$

記述を簡単にするため、減衰固有角周波数 $\bar{\omega}_n$（一般には $\omega_d$ とも書かれる）を以下のように定義する。

$\bar{\omega}_n := \omega_n\sqrt{1 - \zeta^2}$

これを用いると、指数関数部分は $e^{p_1 t} = e^{-\zeta\omega_n t} e^{j\bar{\omega}_n t}$ と書ける。オイラーの公式 $e^{j\theta} = \cos\theta + j\sin\theta$ を用いて式(4.19)を展開すると、以下のようになる。

$\begin{aligned}
y(t) = K \Bigg( 1 &- \frac{\zeta + j\sqrt{1-\zeta^2}}{2j\sqrt{1-\zeta^2}} e^{-\zeta\omega_n t}(\cos\bar{\omega}_n t + j\sin\bar{\omega}_n t) \\
&+ \frac{\zeta - j\sqrt{1-\zeta^2}}{2j\sqrt{1-\zeta^2}} e^{-\zeta\omega_n t}(\cos\bar{\omega}_n t - j\sin\bar{\omega}_n t) \Bigg)
\end{aligned}$

三角関数への整理共通因数 $e^{-\zeta\omega_n t}$ をくくり出し、$\cos$ の項と $\sin$ の項に分けて整理する。複素共役な項の和となっているため、虚数単位 $j$ は消去される。最終的に以下の形にまとまる。

#### $y(t) = K \left( 1 - e^{-\zeta\omega_n t} \cos\bar{\omega}_n t - \frac{\zeta}{\sqrt{1-\zeta^2}}e^{-\zeta\omega_n t} \sin\bar{\omega}_n t \right) \quad \cdots (4.20)$

あるいは、三角関数の合成を行うと以下のようにも表現できる。

$y(t) = K \left( 1 - \frac{e^{-\zeta\omega_n t}}{\sqrt{1-\zeta^2}} \sin(\bar{\omega}_n t + \phi) \right), \quad \phi = \tan^{-1}\frac{\sqrt{1-\zeta^2}}{\zeta}$

この式より、$0 < \zeta < 1$ のときは $\sin, \cos$ が含まれるため振動的な振る舞いとなり、時間が経過すると指数関数 $e^{-\zeta\omega_n t}$ の効果で振幅が $0$ に収束し、最終的に $K$ に落ち着くことがわかる。

--------------------------------------------------------------------------------------------------------------

### 最大値（オーバーシュート）の導出

応答の最大値 $y_{\max}$ を求めるために、時間微分 $\dot{y}(t)$ を計算する。

#### ステップ1：時間微分式(4.20)を $t$ で微分する。

積の微分公式 $(uv)' = u'v + uv'$ を適用し、整理すると $\cos$ の項が相殺され、$\sin$ の項のみが残る。

#### $\dot{y}(t) = \frac{K\omega_n}{\sqrt{1-\zeta^2}} e^{-\zeta\omega_n t} \sin\bar{\omega}_n t \quad \cdots (4.21)$

#### ステップ2：行き過ぎ時間 

#### $T_p$のとき$y(t)$ が極値（最大値）をとるとき、傾きはゼロになる。すなわち $\dot{y}(t) = 0$ 。

式(4.21)において $\sin\bar{\omega}_n t = 0$ となる最小の正の時刻 $t$ を求める。

#### $\bar{\omega}_n t = \pi \quad \Rightarrow \quad T_p = \frac{\pi}{\bar{\omega}_n}$

$\bar{\omega}_n = \omega_n\sqrt{1-\zeta^2}$ を戻して、

#### $T_p = \frac{\pi}{\omega_n\sqrt{1-\zeta^2}} \quad \cdots (4.22)$

#### この時刻 $T_p$ を「行き過ぎ時間」と呼ぶ。

#### ステップ3：最大値 $y_{\max}$求めた $T_p$ を元の式(4.20)に代入して最大値を求める。

ここで、$\sin(\bar{\omega}_n T_p) = \sin(\pi) = 0$、$\cos(\bar{\omega}_n T_p) = \cos(\pi) = -1$ であることを利用すると、計算が大幅に簡単になる。

$\begin{aligned}
y_{\max} = y(T_p) &= K \left( 1 - e^{-\zeta\omega_n T_p} \cdot (-1) - 0 \right) \\
&= K(1 + e^{-\zeta\omega_n T_p}) \quad \cdots (4.23)
\end{aligned}$

--------------------------------------------------------------------------------------------------------------

### 無減衰（$\zeta = 0$）の場合

最後に、減衰係数が $\zeta = 0$ の場合を考える。

式(4.20)において $\zeta \to 0$ とすると、指数関数部分は $e^0 = 1$ となり、第3項は係数が $0$ になって消える。

#### $y(t) = K(1 - \cos\omega_n t) \quad \cdots (4.24)$

この場合、減衰させる要素（指数関数）がなくなるため、振幅 $K$ で永遠に振動し続ける「持続振動」となる。

## 練習問題(1)

<img src="png/4_10.png" alt="1" width="500" height="350">

４．２５ 式の伝達関数のステップ応答を求める。以下のプログラムをShift＋returnで実行すると、伝達関数のプログラム（tfなど）を入力するプロンプトが現れるため、4.25式の伝達関数を入力し、表示されるグラフを確認しなさい

・　下記プログラムを実行「 実行：shift + return 」

・　適切な形式で入力する

・  例） P = 「　tfで書いてね（例: tf([1, 1], [1, 1]) 」など　　　

準備ができたら、下の「from control import tf …」に移動し、 Shift + retrunを押してください。

正解したら、練習問題(2)に移り Shift + retrun を押して進めてください。

In [ ]:
from control import tf, step_response
import numpy as np
import matplotlib.pyplot as plt
from ans.answer1 import get_correct_step_response
from plot_utils import plot_set

def check_step_response_plot():
    print("伝達関数のステップ応答を入力してください。")
    
    try:
        # ユーザーに入力してもらう
        P1 = eval(input("P = "))  # tfで書いてね（例: tf([1,3],[1,3,2])）

        # プロット
        fig, ax = plt.subplots(figsize=(3, 2.3))
        y, t = step_response(P1, np.arange(0, 10, 0.01))
        ax.plot(t, y)
        plot_set(ax, 't', 'y')
        plt.show()

        # 正誤判定
        correct_y, correct_t = get_correct_step_response()
        if np.allclose(y, correct_y, atol=1e-2) and np.allclose(t, correct_t, atol=1e-2):
            print(" 正解！")
        else:
            print(" 不正解です。")

    except Exception as e:
        print("⚠️ エラー:", e)

# 実行
check_step_response_plot()

## 練習問題(2)

<img src="png/4_11.png" alt="1" width="500" height="350">

４．２6 式の伝達関数のステップ応答を求める。以下のプログラムをShift＋returnで実行すると、伝達関数のプログラム（tfなど）を入力するプロンプトが現れるため、4.25式の伝達関数を入力し、表示されるグラフを確認しなさい

・　下記プログラムを実行「 実行：shift + return 」

・　適切な形式で入力する

・  例） P = 「　tfで書いてね（例: tf([1, 1], [1, 1]) 」など　　　

準備ができたら、下の「from control import tf …」に移動し、 Shift + retrunを押してください。

正解したら、また「状態空間モデルの時間応答」から Shift + retrun を押して進めていってください。

In [ ]:
from control import tf, step_response
import numpy as np
import matplotlib.pyplot as plt
from ans.answer2 import get_correct_step_response
from plot_utils import plot_set

def check_step_response_plot():
    print("伝達関数のステップ応答を入力してください")
    
    try:
        # ユーザーに入力してもらう
        P2 = eval(input("P2 = "))  # tfで書いてね（例: tf([0,1],[1,2,2,1])）

        # プロット
        fig, ax = plt.subplots(figsize=(3, 2.3))
        y, t = step_response(P2, np.arange(0, 15, 0.01))
        ax.plot(t, y)
        plot_set(ax, 't', 'y')
        plt.show()

        # 正誤判定
        correct_y, correct_t = get_correct_step_response()
        if np.allclose(y, correct_y, atol=1e-2) and np.allclose(t, correct_t, atol=1e-2):
            print(" 正解です！")
        else:
            print(" 不正解です。 ")

    except Exception as e:
        print("⚠️ エラー:", e)

# 実行
check_step_response_plot()


# 4.2　状態空間モデルの時間応答　p.107

つぎは状態空間モデルで時間応答を考えてみよう。

伝達関数モデルでは、入力に対して出力がどのように変化するかに注目していたが、状態空間モデルでは、それに加えて初期値が出力に与える影響も考えることができる。

そこで、入力を $u=0$ とした $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t)$ の振る舞いを考えてみる。

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

Pythonでは、初期値応答は initial 関数を使うと調べることができる。

#### 「　x, t = initial(sys, Td, X0)　」

引数は sys, Td, X0 で、sys と Td は step のときと同じ。

X0 は初期状態で、sys の状態の数だけ与える。

たとえば、X0 = [1, 2] のようにリストで渡す。なお、初期状態を 0 とする場合は、省略することができる。

initial の返り値は、x と t で、x が状態の応答、t が時間。なお、多次元の x の各要素を参照する場合は、スライスを用いて x[:,0], x[:,1] のようにする。

以下では、

$\boldsymbol{A} = \begin{bmatrix} 0 & 1 \\ -4 & -5 \end{bmatrix}, \quad \boldsymbol{B} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \quad \boldsymbol{C} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}, \quad \boldsymbol{D} = \begin{bmatrix} 0 \\ 0 \end{bmatrix} \quad \cdots (4.27)$

で与えられるシステムを考える。ただし、状態 $\boldsymbol{x}$ の振る舞いを調べるために、$\boldsymbol{C}$ を単位行列としている（つまり、$y = x$）。

#### 図10　状態空間モデルの初期値応答

In [15]:
# 制御システムに関連するライブラリから必要な関数をインポート
from control.matlab import ss, step, initial, lsim

# ss: 状態空間モデルを作成するための関数
# step: システムのステップ応答を計算するための関数
# initial: 初期状態からのシステムの応答を計算するための関数
# lsim: 任意の入力に対するシステムの応答を計算するための関数

In [16]:
# 状態空間モデルの各行列を定義
A = [[0, 1], [-4, -5]]  # A行列（システムの状態遷移行列）
B = [[0], [1]]  # B行列（入力行列）
C = np.eye(2)  # C行列（出力行列、2x2の単位行列）
D = np.zeros([2, 1])  # D行列（直接伝達行列、2x1の零行列）

# 状態空間モデルを作成
P = ss(A, B, C, D)  # ss()関数で状態空間モデルを生成

In [ ]:
# 時間ベクトルTdを0から5秒まで0.01秒刻みで定義
Td = np.arange(0, 5, 0.01)
# 初期状態X0を[-0.3, 0.4]に設定
X0 = [-0.3, 0.4]

# ゼロ入力応答を計算。初期状態X0からのシステムの挙動を取得
x, t = initial(P, Td, X0)

# グラフを作成
fig, ax = plt.subplots(figsize=(3, 2.3))

# システムの状態x1をプロット
ax.plot(t, x[:, 0], label = '$x_1$')
# システムの状態x2をプロット（破線で表示）
ax.plot(t, x[:, 1], ls = '-.', label = '$x_2$')
# x軸の目盛りを0から5まで等間隔に設定
ax.set_xticks(np.linspace(0, 5, 6))
# y軸の目盛りを-0.4から0.6まで等間隔に設定
ax.set_yticks(np.linspace(-0.4, 0.6, 6))

# 軸ラベルと凡例を設定するためのカスタム関数を呼び出し
# 軸ラベルと凡例を設定するためのカスタム関数を呼び出し
plot_set(ax, 't', 'x')

# 凡例の位置を設定
ax.legend(loc='best')

状態 $\boldsymbol{x} = [x_1 \ x_2]^\top$ は 2次元なので、図4.10 には 2 本の線がプロットされている。すなわち、初期値 $x_1(0) = -0.3, \ x_2(0) = 0.4$ からスタートして、最終的にどちらも 0 に収束している。

ところで、図 3.3（62 ページ）の垂直駆動アームは、初期角度と初期角速度を与えてスタートさせたとき、何も入力がなければ、粘性摩擦と重力の影響を受けて、最終的に鉛直下向きに垂れ下がった状態で静止する。

つまり、$x_1(t) \to 0, \ x_2(t) \to 0 \ (t \to \infty)$ となる。上記の例題はこれに対応している。

## 行列指数関数の計算

微分方程式 $\dot{x}(t) = ax(t)$ の解は、

#### $x(t) = e^{at}x(0) \quad \cdots (4.28)$

である。

この $a$ が行列 $\boldsymbol{A}$ に置き換わった状態方程式 $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t)$ の解は、

#### $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{x}(0) \quad \cdots (4.29)$

となる。ただし、$e^{\boldsymbol{A}t}$ は 行列指数関数（状態遷移行列） で、

$\begin{aligned}
e^{\boldsymbol{A}t} &= \boldsymbol{I} + \boldsymbol{A}t + \frac{\boldsymbol{A}^2t^2}{2!} + \frac{\boldsymbol{A}^3t^3}{3!} + \cdots \\
&= \mathcal{L}^{-1} \left[ (s\boldsymbol{I} - \boldsymbol{A})^{-1} \right] \quad \cdots (4.30)
\end{aligned}$

となる。ただし、$\boldsymbol{I}$ は単位行列である。

この行列指数関数が計算できれば、状態方程式の振る舞いを求めることができる。なお、行列指数関数は、式 (4.30) の逆ラプラス変換を用いる方法により直接手計算で求めることもできるが、つぎのように Python を用いて求めることもできる。

In [ ]:
import sympy as sp # 数式の簡略化、展開、微積分、ラプラス変換など、数式の操作をシンボリックに実行することができる。
import numpy as np # 数値計算において、特に行列の計算や多次元配列の操作を効率的に行いたい場合に使用する。
# sp.init_printing()  # シンボリック計算の結果をきれいに表示

# システム行列 A を定義
A = np.array([[0, 1],[-4, -5]])
# シンボリックな変数を定義
s = sp.Symbol('s')  # ラプラス変換の変数 s
t = sp.Symbol('t', positive=True)  # 時間変数 t
# 行列 G = sI - A を計算（s はスカラー、I は単位行列）
G = s*sp.eye(2) - A

# 行列 G の逆行列を求め、そのラプラス逆変換を計算
exp_At = sp.inverse_laplace_transform(sp.simplify(G.inv()), s, t)

# 結果を表示
exp_At

また、ある時刻における行列指数関数の値は、以下で求めることができる。

In [ ]:
# scipyのlinalgモジュールからexpmをインポート
from scipy.linalg import expm
# 数値計算を行うためにnumpyをインポート
import numpy as np

# 状態遷移行列 A の定義
A = np.array([[0, 1], [-4, -5]])
# 時間 t を設定（ここでは t = 5）
t = 5

# 行列指数関数 e^(At) を計算
result = expm(A * t)

# 結果を表示
result

## 零入力応答と零状態応答 p.110

状態方程式 $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t)$ の解は、

#### $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{x}(0) + \int_0^t e^{\boldsymbol{A}(t-\tau)} \boldsymbol{B}u(\tau) \,d\tau \quad (t \ge 0) \quad \cdots (4.31)$

となる。式 (4.31) で、第 1 項を 零入力応答、第 2 項を 零状態応答と呼ぶ。

制御システムや動的システムにおいて、初期状態がゼロ。システムの最初の状態がすべてゼロの状態で、外部入力に対するシステムの応答を指す。

・状態空間での計算: システムの状態方程式を解き、初期状態がゼロの場合の解を求めることで、零状態応答が得られる。

・伝達関数での計算: 入力に対する伝達関数の出力を計算し、それをラプラス逆変換して時刻領域での応答を得ることができる。

-------------------------------------------------------------------------------------------------------------------------------------

ここでは、初期値を $\boldsymbol{x}(0) = [0 \ \ 0]^\top$ とし、入力を $u(t) = 1 \ (t \ge 0)$ （ステップ入力）としたときの応答（零状態応答）

#### $\boldsymbol{x}(t) = \int_0^t e^{\boldsymbol{A}(t-\tau)} \boldsymbol{B} \,d\tau \quad \cdots (4.32)$

をみてみよう。

### 図11 状態空間モデルの零状態応答

In [ ]:
import numpy as np  # 数値計算のためのライブラリ 数値計算において、特に行列の計算や多次元配列の操作を効率的に行いたい場合に使用する。
import matplotlib.pyplot as plt  # プロットのためのライブラリ
from control.matlab import ss, step  # 制御システムの状態空間表現とステップ応答を計算するための関数

# システムの定義 (状態空間表現)
A = [[0, 1], [-4, -5]]
B = [[0], [1]]
C = np.eye(2)  # 2x2の単位行列
D = np.zeros([2, 1])  # 2x1の零行列
P = ss(A, B, C, D)  # 状態空間モデルを作成

# 時間の離散的な点を作成 (0から5秒まで、0.01秒間隔)
Td = np.arange(0, 5, 0.01)

# ステップ応答を計算 (ゼロ状態応答)
x, t = step(P, Td)

# プロットの設定
fig, ax = plt.subplots(figsize=(3, 2.3))  # グラフのサイズを設定
ax.plot(t, x[:, 0], label='$x_1$')  # x1のプロット
ax.plot(t, x[:, 1], ls='-.', label='$x_2$')  # x2のプロット (点線)

# X軸の目盛りを設定 (0から5までを6等分)
ax.set_xticks(np.linspace(0, 5, 6))

# Y軸の目盛りを設定 (-0.4から0.6までを6等分)
ax.set_yticks(np.linspace(-0.4, 0.6, 6))

# 軸ラベルと凡例を設定するためのカスタム関数を呼び出し
# ここで't'は時間、'x'は状態ベクトル、凡例の位置を'best'に設定
def plot_set(ax, xlabel, ylabel, loc='best'):
    """
    プロットのラベルと凡例の設定を行う関数
    ax: グラフの軸
    xlabel: x軸のラベル
    ylabel: y軸のラベル
    loc: 凡例の位置 ('best' は自動で最適な位置)
    """
    ax.set_xlabel(xlabel)  # x軸のラベルを設定
    ax.set_ylabel(ylabel)  # y軸のラベルを設定
    ax.legend(loc=loc)     # 凡例の位置を設定

plot_set(ax, 't', 'x')  # 位置引数を3つに変更

また、lsim 関数を使って、式 (4.31) の時間応答を確認する。先ほど調べた初期値応答（零入力応答）と零状態応答を足し合わせたものになる。

### 「　y, t, x = lsim(sys, Ud, Td, X0)　」

引数は sys, Ud, Td, X0 で、sys, Td, X0 は initial のときと同じで、Ud は入力信号。

たとえば、Td = np.arange(0, 5, 0.01) としたとき、Ud = 1 * (Td >= 0) とすれば、ステップ入力を指定することができる。

lsim の返り値は y, t, x で、y がシステムの出力の応答、t が時間、x が状態の応答。

システムの $\boldsymbol{C}$ 行列を単位行列とする場合は、y は状態 x と同じになる。

具体的に、「状態空間モデルの時間応答」 を実行すると、図 4.12 が得られる。なお、「状態空間モデルの時間応答」 において、関数の返り値で不要なものは、アンダースコアを用いて無視している。

### 図12 状態空間モデルの時間応答

In [ ]:
import numpy as np # 数値計算のためのライブラリ 数値計算において、特に行列の計算や多次元配列の操作を効率的に行いたい場合に使用する。
import matplotlib.pyplot as plt  # グラフ描画のためのライブラリ
from control.matlab import ss, step, initial, lsim  # 制御システムの動作をシミュレートするための関数群


# 時間ベクトルとステップ入力の定義
Td = np.arange(0, 5, 0.01)
Ud = 1*(Td>=0)  # ステップ入力
X0 = [-0.3, 0.4]  # 初期状態ベクトル

# システムの応答を取得
xst, t = step(P, Td)  # ゼロ状態応答
xin, _ = initial(P, Td, X0)  # ゼロ入力応答
x, _, _ = lsim(P, Ud, Td, X0)  # 総合応答（ゼロ入力とゼロ状態の両方）

# プロットの準備
fig, ax = plt.subplots(1, 2, figsize=(6, 2.3))

# 各状態変数のプロット
for i in [0, 1]:
    # 各状態変数の応答をプロット
    ax[i].plot(t, x[:, i], label='response')  # 総合応答
    ax[i].plot(t, xst[:, i], ls='--', label='zero state')  # ゼロ状態応答
    ax[i].plot(t, xin[:, i], ls='-.', label='zero input')  # ゼロ入力応答
    
    # 軸の目盛りを設定
    ax[i].set_xticks(np.linspace(0, 5, 6))
    ax[i].set_yticks(np.linspace(-0.4, 0.6, 6))

# 軸ラベルと凡例を設定するためのカスタム関数を呼び出し
plot_set(ax[0], 't', '$x_1$')  # 左側プロット（x1軸）
plot_set(ax[1], 't', '$x_2$', 'best')  # 右側プロット（x2軸）

# プロットを自動でレイアウト調整
fig.tight_layout()

## 状態方程式の解の導出 P.112

#### 1. 斉次方程式（入力なし）の場合

まず、 $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t)$ の解が、以下の形になることを確かめる。

#### $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{x}(0) \quad \cdots (4.33)$

微分の計算式(4.33)が本当に解であるかを確認するため、両辺を時間 $t$ で微分する。ここで、行列指数関数 $e^{\boldsymbol{A}t}$ の微分公式 $\frac{d}{dt}e^{\boldsymbol{A}t} = \boldsymbol{A}e^{\boldsymbol{A}t} = e^{\boldsymbol{A}t}\boldsymbol{A}$ を用いる。

$\begin{aligned}
\text{左辺} &= \frac{d}{dt}\boldsymbol{x}(t) \\
&= \frac{d}{dt} \left( e^{\boldsymbol{A}t}\boldsymbol{x}(0) \right) \\
&= \left( \frac{d}{dt} e^{\boldsymbol{A}t} \right) \boldsymbol{x}(0) \quad (\because \boldsymbol{x}(0)\text{は定数ベクトル}) \\
&= \boldsymbol{A} e^{\boldsymbol{A}t} \boldsymbol{x}(0) \quad \cdots (4.34)
\end{aligned}$

ここで、$e^{\boldsymbol{A}t}\boldsymbol{x}(0)$ は元の $\boldsymbol{x}(t)$ そのものなので、

#### $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) \quad \cdots (4.35)$

となり、元の微分方程式と一致する。よって、式(4.33)は確かに解であることがわかる。

---------------------------------------------------------------------------------------------------------------------------------------------------------------

### 2. 非斉次方程式（入力あり）の場合


次に、入力項がある一般的な状態方程式 $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t)$ の解を求める。

ここでは「定数変化法」の応用として、解の候補を以下のように仮定する。これは $e^{\boldsymbol{A}t}$ という係数はそのままに、定数ベクトル $\boldsymbol{x}(0)$ の部分を未知の時間関数 $\boldsymbol{z}(t)$ に置き換えたもの。

#### $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{z}(t) \quad (\text{解の候補})$

ただし、初期時刻 $t=0$ では $\boldsymbol{x}(0) = e^{\boldsymbol{O}}\boldsymbol{z}(0) = \boldsymbol{I}\boldsymbol{z}(0) = \boldsymbol{z}(0)$ なので、$\boldsymbol{z}(0) = \boldsymbol{x}(0)$ 。

### ・手順1：候補式を微分するこの仮定した式を時間微分する。

積の微分公式 $(fg)' = f'g + fg'$ を行列とベクトルに適用する。

#### $\begin{aligned}
\dot{\boldsymbol{x}}(t) &= \frac{d}{dt} \left( e^{\boldsymbol{A}t}\boldsymbol{z}(t) \right) \\
&= \left( \frac{d}{dt} e^{\boldsymbol{A}t} \right) \boldsymbol{z}(t) + e^{\boldsymbol{A}t} \left( \frac{d}{dt} \boldsymbol{z}(t) \right) \\
&= \boldsymbol{A}e^{\boldsymbol{A}t}\boldsymbol{z}(t) + e^{\boldsymbol{A}t}\dot{\boldsymbol{z}}(t)
\end{aligned}$

ここで、仮定より $e^{\boldsymbol{A}t}\boldsymbol{z}(t) = \boldsymbol{x}(t)$ なので、第1項を置き換える。

#### $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + e^{\boldsymbol{A}t}\dot{\boldsymbol{z}}(t) \quad \cdots (4.36)$

### ・手順2：元の状態方程式と比較する

この式(4.36)と、元の状態方程式 $\dot{\boldsymbol{x}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t)$ を比較する。

これらが一致するためには、余分な項同士が等しくなければならない。

#### $\boldsymbol{A}\boldsymbol{x}(t) + e^{\boldsymbol{A}t}\dot{\boldsymbol{z}}(t) = \boldsymbol{A}\boldsymbol{x}(t) + \boldsymbol{B}u(t)$

両辺から $\boldsymbol{A}\boldsymbol{x}(t)$ を引くと、以下の関係式が得られる。

#### $e^{\boldsymbol{A}t}\dot{\boldsymbol{z}}(t) = \boldsymbol{B}u(t) \quad \cdots (4.37)$

### ・手順3：$\boldsymbol{z}(t)$ を求める

式(4.37)から $\dot{\boldsymbol{z}}(t)$ を求めるために、両辺に左から逆行列 $(e^{\boldsymbol{A}t})^{-1} = e^{-\boldsymbol{A}t}$ を掛ける。

#### $\begin{aligned}
e^{-\boldsymbol{A}t} \cdot e^{\boldsymbol{A}t}\dot{\boldsymbol{z}}(t) &= e^{-\boldsymbol{A}t}\boldsymbol{B}u(t) \\
\boldsymbol{I}\dot{\boldsymbol{z}}(t) &= e^{-\boldsymbol{A}t}\boldsymbol{B}u(t) \\
\dot{\boldsymbol{z}}(t) &= e^{-\boldsymbol{A}t}\boldsymbol{B}u(t)
\end{aligned}$

この両辺を $0$ から $t$ まで積分する（積分変数を $\tau$ とする）。

#### $\begin{aligned}
\int_0^t \dot{\boldsymbol{z}}(\tau) d\tau &= \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau \\
\left[ \boldsymbol{z}(\tau) \right]_0^t &= \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau \\
\boldsymbol{z}(t) - \boldsymbol{z}(0) &= \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau
\end{aligned}$

$\boldsymbol{z}(0) = \boldsymbol{x}(0)$ であったことを思い出し、移項すると $\boldsymbol{z}(t)$ が求まる。

#### $\boldsymbol{z}(t) = \boldsymbol{x}(0) + \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau \quad \cdots (4.38)$

### ・手順4：$\boldsymbol{x}(t)$ に戻す

最後に、求まった $\boldsymbol{z}(t)$ を、最初の解の候補式 $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{z}(t)$ に代入する。

#### $\begin{aligned}
\boldsymbol{x}(t) &= e^{\boldsymbol{A}t} \left( \boldsymbol{x}(0) + \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau \right) \\
&= e^{\boldsymbol{A}t}\boldsymbol{x}(0) + e^{\boldsymbol{A}t} \int_0^t e^{-\boldsymbol{A}\tau}\boldsymbol{B}u(\tau) d\tau
\end{aligned}$

ここで、第2項の $e^{\boldsymbol{A}t}$ は積分変数 $\tau$ に無関係な定数係数とみなせるため、積分の内側に入れることができる。行列指数の性質 $e^{\boldsymbol{A}t}e^{-\boldsymbol{A}\tau} = e^{\boldsymbol{A}(t-\tau)}$ を利用してまとめます。

#### $\boldsymbol{x}(t) = e^{\boldsymbol{A}t}\boldsymbol{x}(0) + \int_0^t e^{\boldsymbol{A}(t-\tau)}\boldsymbol{B}u(\tau) d\tau \quad \cdots (4.39)$

これで状態方程式の一般解が導出される。

# 4.3　安定性　p.114

### 入出力安定性

あるシステムにおいて、「入力が有界なら出力も必ず有界である」という性質を指す。

1 次遅れ系や 2 次遅れ系のステップ応答を調べた際、パラメータの選び方次第で、出力応答が発散することがあった。これはシステムが不安定であることを意味する。そこで、システムの安定性についてもう少し詳しく考えてみる。

#### 有界な信号を入力としたときに出力も有界になることを入出力安定や BIBO 安定 (bounded input, bounded output stability : 有界入力・有界出力安定)、または単に安定という。

ちなみに有界な信号とは、無限大に発散しない信号のことで、つぎのように定義さる。

#### $|u(t)| \leq M < \infty, \quad \forall t, \quad \exists M > 0 \quad \cdots (4.40)$

つまり、1 次遅れ系や 2 次遅れ系ではパラメータによって入出力安定でなくなる場合があるということになる。

では、一般的に、いつ不安定になるのか。これは、伝達関数の 極 ($P(s) = \infty$ となる $s$, 分母多項式の根) を調べることでわかる。

具体的に安定な場合と、不安定な場合、それぞれについて極をみてみる。

Python で伝達関数の極を調べるためには、poles 関数を使う。具体的に、sys.poles() のようにメソッドとして実行することで、極が取得できる。また、poles をインポートして、p = poles(sys) のように引数にモデルを指定することで、同様の結果が得られる。1 次遅れ系で、$K=1, T=1, -1$ としたときの伝達関数の極を確認する。

$\mathcal{P}_1$ の極は $-1$, $\mathcal{P}_2$ の極は $1$ 。つぎに、2 次遅れ系で $K=1、 \omega_n=1、 \zeta=0.025、 -0.025$ としたときの伝達関数の極を求める。

## 極

システムの伝達関数の分母の根であり、システムの応答や安定性を決定する重要な要素。

In [22]:
from control.matlab import tf, tfdata

In [ ]:
# 必要な関数のインポート
from control.matlab import tf

# 伝達関数 P1: 安定な1次遅れ系（極は -1 にあり、負の実数 ⇒ 安定）
P1 = tf([0, 1], [1, 1])
print('P1:', P1.poles())  # P1 の極（ポール）を表示

# 伝達関数 P2: 不安定な1次遅れ系（極は +1 にあり、正の実数 ⇒ 不安定）
P2 = tf([0, 1], [-1, 1])
print('P2:', P2.poles())  # P2 の極を表示

# 伝達関数 P3: 複素共役の安定な極（実部が負 ⇒ 減衰振動を伴う安定系）
P3 = tf([0, 1], [1, 0.05, 1])
print('P3:', P3.poles())  # P3 の極を表示

# 伝達関数 P4: 複素共役の不安定な極（実部が正 ⇒ 増幅する振動 ⇒ 不安定）
P4 = tf([0, 1], [1, -0.05, 1])
print('P4:', P4.poles())  # P4 の極を表示

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))

P1pole = P1.poles()
P2pole = P2.poles()
P3pole = P3.poles()
P4pole = P4.poles()
ax.scatter(P1pole.real, P1pole.imag, s=50, marker='o',label='$P_1$', color='k')
ax.scatter(P2pole.real, P2pole.imag, s=50, marker='^',label='$P_2$', color='k')
ax.scatter(P3pole.real, P3pole.imag, s=50, marker='x',label='$P_3$', color='k')
ax.scatter(P4pole.real, P4pole.imag, s=50, marker='*',label='$P_4$', color='k')

ax.set_xlim(-1.2,1.2)
ax.set_ylim(-1.2,1.2)
plot_set(ax, 'Re', 'Im', 'best')

<img src="png/4_7.png" alt="1" width="600" height="450">

安定な場合は、極の実部が負になっており、複素平面において左半平面に極が存在していることがわかる。実際、システムの入出力安定性に関して、つぎの結果が知られている。

システムが入出力安定であるための必要十分条件は，伝達関数のすべての極の実部が負であることである．

すなわち、伝達関数の極が複素平面の左半平面内にあれば、システムは入出力安定となる。したがって、伝達関数の極を調べれば、安定性が判別できる。

#### なお、実部が負の極を 安定極 といい、そうでない極を 不安定極 という。

Python では、極は上記のように poles を用いて求めることができるほか、つぎのように、roots を使って分母多項式の根を求めることもできる。

伝達関数 P4 の分母係数を取り出し、その根（= 極 / ポール）を求める

In [ ]:
# 伝達関数 P4 の分母係数を取得
_, [[Dp]] = tfdata(P4)

# 分母係数（多項式の係数）を表示
print(Dp)

# 分母係数 Dp の多項式の根（極）を求めて表示
# np.roots は多項式の解（極）を求める関数
print(np.roots(Dp))

#### 分母多項式の根を伝達関数の極といったが、分子多項式の根 (P(s)=0となる;)のことを零点と呼ぶ。

この零点もシステムの応答 を決めるものだが、基本的にシステムを不安定にするものではない。

Python では、零点は関数 zeros を用いて調べることができる(もちろん分 子多項式の根を求めても同じ結果が得られる)。

## 4.3.2　漸近安定性　p.118

システムの応答が時間が経つにつれて安定した状態に収束する特性を指す。

漸近安定：システムのすべての極が左半平面にある場合（すなわち、すべての極の実部が負である場合）、システムは漸近安定である。この場合、時間が無限大に近づくとシステムの応答はゼロに収束する。

漸近不安定：システムの極の中に右半平面にあるもの（実部が正）や、虚軸上にあるもの（実部がゼロ）が含まれている場合、そのシステムは漸近不安定と見なす。この場合、システムの応答は時間とともに発散したり、一定の値に収束せず、安定しない。

状態空間モデルの場合、安定性はシステムの $\boldsymbol{A}$ 行列の 固有値 を調べればわかる。システムが安定であるための必要十分条件は，行列 $\boldsymbol{A}$ のすべての固有値の実部が負であることである．

#### ただし、ここでの安定性は、$$\lim_{t \to \infty} \boldsymbol{x}(t) = 0 \quad \cdots (4.41)$$となることで、このような安定性を 漸近安定性 という。

とくに、状態空間モデルが漸近安定であれば、有界な入力に対して出力は有界（入出力安定，BIBO 安定）となる。これは、行列 $\boldsymbol{A}$ の特性多項式と伝達関数の分母多項式が同じであることからわかる。しかし逆は必ずしも正しくないことに注意してください。

---------------------------------------------------------------------------------------------------------------------------------------------------------------

### 位相面図

下記のコード「位相面図」 を利用して、状態 $x_1$ と $x_2$ の 位相面図 (本書では、状態 $\boldsymbol{x}$ の軌道を $x_1 - x_2$ 平面にプロットしたものをこのように呼びます) を描くと 図 4.14 (p.119) のようになる。 

このグラフから任意の初期値からスタートしたとき、必ず原点に収束する軌跡が描かれていることが確認できる。

図中の直線は、行列 $\boldsymbol{A}$ の固有値に対応する固有ベクトルをプロットしたもの。 上記の例の場合、固有値は、$-1$ と $-4$ なので、それらに対応する固有ベクトルが破線で描かれている。なお、固有ベクトルは、$\dot{\boldsymbol{x}} = \boldsymbol{A}\boldsymbol{x} = \lambda \boldsymbol{x}$ を満たすものなので、この直線の上に一度乗ってしまえば、後は、その直線に沿って状態が遷移する。

#### この直線のことを 不変空間 と呼ぶ。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# wを1.5に設定して、XとYの範囲を-1.5から1.5まで100分割する
w = 1.5
Y, X = np.mgrid[-w:w:100j, -w:w:100j]

# 行列Aを定義し、その固有値と固有ベクトルを計算
A = np.array([[0, 1], [-4, -5]])
s, v = np.linalg.eig(A)

# 固有値を表示
print('固有値=', s)

# 固有ベクトルを使って流れ場（U, V）を計算
U = A[0, 0] * X + A[0, 1] * Y
V = A[1, 0] * X + A[1, 1] * Y

# 時間軸の値（t）は-1.5から1.5まで0.01ステップで作成
t = np.arange(-1.5, 1.5, 0.01)

# プロットを作成
fig, ax = plt.subplots(figsize=(3, 3))

# 固有空間のプロット
if s.imag[0] == 0 and s.imag[1] == 0:  # 固有値が実数の場合
    # 固有ベクトル1のプロットを追加
    ax.plot(t, (v[1, 0] / v[0, 0]) * t, ls='--', color='k', lw=2, label='$Eigenvector 1$')  # ラベルを追加
    # 固有ベクトル2のプロットを追加
    ax.plot(t, (v[1, 1] / v[0, 1]) * t, ls='--', color='k', lw=2, label='$Eigenvector 2$')  # ラベルを追加

# ベクトル場のストリームプロットを描画
# X, Y で定義したグリッドに対して、U, V の流れ場を表示
ax.streamplot(X, Y, U, V, density=0.7, color='k', linewidth=0.5)

# 軸の範囲を設定
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

# 軸の目盛りを設定
ax.set_xticks(np.arange(-1.5, 1.51, step=0.5))
ax.set_yticks(np.arange(-1.5, 1.51, step=0.5))

# プロットのラベルを設定（カスタム関数を利用）
plot_set(ax, '$x_1$', '$x_2$')

# 凡例を表示
# labelで指定した内容がプロットに表示されます
ax.legend(loc='best')

## 練習問題(p.120)　

<img src="png/4_13.png" alt="1" width="600" height="450">

回答コードを載せておきます

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# wを1.5に設定して、XとYの範囲を-1.5から1.5まで100分割する
w = 1.5
Y, X = np.mgrid[-w:w:100j, -w:w:100j]

# 行列Aを定義し、その固有値と固有ベクトルを計算
A = np.array([[0, 1], [-4, 5]])
s, v = np.linalg.eig(A)

# 固有値を表示
print('固有値=', s)

# 固有ベクトルを使って流れ場（U, V）を計算
U = A[0, 0] * X + A[0, 1] * Y
V = A[1, 0] * X + A[1, 1] * Y

# 時間軸の値（t）は-1.5から1.5まで0.01ステップで作成
t = np.arange(-1.5, 1.5, 0.01)

# プロットを作成
fig, ax = plt.subplots(figsize=(3, 3))

# 固有空間のプロット
if s.imag[0] == 0 and s.imag[1] == 0:  # 固有値が実数の場合
    # 固有ベクトル1のプロットを追加
    ax.plot(t, (v[1, 0] / v[0, 0]) * t, ls='--', color='k', lw=2, label='$Eigenvector 1$')  # ラベルを追加
    # 固有ベクトル2のプロットを追加
    ax.plot(t, (v[1, 1] / v[0, 1]) * t, ls='--', color='k', lw=2, label='$Eigenvector 2$')  # ラベルを追加

# ベクトル場のストリームプロットを描画
# X, Y で定義したグリッドに対して、U, V の流れ場を表示
ax.streamplot(X, Y, U, V, density=0.7, color='k', linewidth=0.5)

# 軸の範囲を設定
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

# 軸の目盛りを設定
ax.set_xticks(np.arange(-1.5, 1.51, step=0.5))
ax.set_yticks(np.arange(-1.5, 1.51, step=0.5))

# プロットのラベルを設定（カスタム関数を利用）
ax.set_xlabel('$x_1$')
ax.set_ylabel('$x_2$')

# 凡例を表示
# labelで指定した内容がプロットに表示されます
ax.legend(loc='best')

## リアプノフの安定判別法　p.121

線形・非線形システムの安定性を解析するための方法


特に「微分方程式で表される動的システム」が安定かどうかを判断できる。

A：システムの係数行列（定数行列）

Q = Q^T > 0 ： 任意の対称正定値行列（入力）

P = P^T > 0 ： 求めたい対称正定値行列（出力）

対称正定値行列 ： 次の2つの条件を満たす実行列Aのことを言う

・対称行列 ： A^T = A  つまり、行列が左右対称になっていること。

・正定値   ： 任意の非ゼロベクトルxに対して、x^T・Ax > 0 が成り立つこと。

---------------------------------------------------------------------------------------------------------------------------------------------------------------

状態方程式の安定性を調べるときには、行列 $\boldsymbol{A}$ の固有値を直接調べるが、それをせずに、リアプノフ関数と呼ばれる関数の性質によって、間接的に安定性を調べる方法がある。

#### これは，リアプノフの安定定理と呼ばれている。

この結果を用いると，行列方程式を解くことでシステムの安定性が判別できる。

システム $\dot{\boldsymbol{x}} = \boldsymbol{A}\boldsymbol{x}$ が漸近安定であることは、以下と等価である。行列 $\boldsymbol{Q} = \boldsymbol{Q}^\top > 0$ が任意に与えられたとき、リアプノフ方程式

#### $\boldsymbol{P}\boldsymbol{A} + \boldsymbol{A}^\top \boldsymbol{P} = -\boldsymbol{Q} \quad (4.42)$

を満たす行列 $\boldsymbol{P} = \boldsymbol{P}^\top > 0$ が一意に存在する。

以下の行列の固有値は、 $-1$ と $-4$ ですので、システムは漸近安定です。

In [28]:
# lyap 関数（リアプノフ方程式を解く関数）をインポート
from control.matlab import lyap

In [ ]:
# システム行列 A を定義（2次の連続時間線形システム）
A = np.array([[0, 1], [-4, -5]])

# 行列 A の固有値を計算し、表示
# 固有値はシステムの安定性や動作の性質（発散・収束・振動など）を判断する際に使われる
print(np.linalg.eigvals(A))

#### Aの固有値が負なので，Aは安定である

In [ ]:
# 2×2の単位行列を Q に設定（Q は対称正定値行列として与える）
Q = np.eye(2)

# リアプノフ方程式 A^T P + P A = -Q を解くため、
# lyap(M, Q) は M P + P M^T = -Q を解くため、
# ここでは M = A.T（Aの転置）を使うことで、リアプノフ方程式に対応
P = lyap(A.T, Q)

# 求めた P の固有値を表示
# P が正定値（固有値がすべて正）であれば、システムは漸近安定であることを示す
print(np.linalg.eigvals(P))

であることから， $\boldsymbol{P}$ が正定であることがわかる。これより，システムが漸近安定であることが確認できる。

なお、 lyap(M, Q) は $\boldsymbol{M}\boldsymbol{P} + \boldsymbol{P}\boldsymbol{M}^\top = -\boldsymbol{Q}$ の解を求める関数であるため、引数に $\boldsymbol{A}$ の転置を与えている。

#### Pの固有値が正なので，Pは正定値行列となり，Aが安定であることがわかる

# 4.4　語句と振る舞いの関係　p.122

伝達関数モデルの極の実部や、状態空間モデルの $\boldsymbol{A}$ 行列の固有値の実部が負であるとき、システムが安定であることがわかった。ここでは、もう一歩踏み込んで、極や固有値とシステムの振る舞いの関係を説明する。

まず、極が負側に大きくなるほど、応答が速くなる。 1 次遅れ系の場合は、極が $s = -\frac{1}{T}$ となるので、$T$ が小さいほど応答が速くなることに対応している。

また 2 次遅れ系の場合には、極が

$s = -\zeta\omega_n \pm \omega_n\sqrt{\zeta^2 - 1}$

なので、$\omega_n$ が大きくなると応答が速くなることに対応している。

つぎに、極の虚部が 0 でないとき、振動的になり、虚部が大きいほど振動が速くなります。 1 次遅れ系の極は実部のみなので、振動しないということがわかる。さらに、 2 次遅れ系では、$\zeta$ の絶対値が 1 より小さいときは、

$s = -\zeta\omega_n \pm j\omega_n\sqrt{1 - \zeta^2}$

となるので、$\zeta$ が 0 に近いほど虚部の大きさが大きくなり振動的になる。また、$\zeta = 0$ のときには、実部が 0 となるが、この場合は、減衰せずに持続振動となる。

以上の説明を整理したものが、図 4.16 (p.123) 。

ここで、 $\times$ 印は、複素平面上の極（固有値）の位置を示している（複素数の場合は、共役複素数のペアで存在する）。

極が右半平面にある場合、すなわち極の実部が正の場合には、発散する。一方、極が左半平面にある場合、すなわち極の実部が負の場合にはある値に収束する。なお、極が虚軸上にある場合、すなわち極の実部が 0 の場合には持続振動となる。それから、極が実部だけ（虚部が 0）の場合には振動せずに収束する。そして、極の実部が負側に大きいほど収束が速くなり、極の虚部の大きさが大きいほど振動周期が短くなる。

<img src="png/4_14.png" alt="1" width="600" height="450">

最後に、零点と振る舞いの関係を簡単に説明する。ここでは、つぎの 3 つの伝達関数を考える。

#### $\begin{aligned}
\mathcal{P}_1(s) &= \frac{\omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2} & (4.43) \\[5pt]
\mathcal{P}_2(s) &= \frac{3s + \omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2} & (4.44) \\[5pt]
\mathcal{P}_3(s) &= \frac{3s - \omega_n^2}{s^2 + 2\zeta\omega_n s + \omega_n^2} & (4.45)
\end{aligned}$

$\mathcal{P}_1$ は零点がない伝達関数。また、$\mathcal{P}_2$ の零点は $s = -\frac{\omega_n^2}{3}$ で負の値となっている。

#### このような零点を 安定零点 という。

対して、$\mathcal{P}_3$ の零点は $s = \frac{\omega_n^2}{3}$ で正の値となる。

#### このような零点を 不安定零点 という。

#### さらに、 $\mathcal{P}_1$ や $\mathcal{P}_2$ のようなシステムを 最小位相系 といい、 $\mathcal{P}_3$ のような不安定零点をもつシステムを 非最小位相系 という。

In [31]:
# tf : 伝達関数（Transfer Function）を作成する関数
# ss : 状態空間（State Space）表現を作成する関数
# lsim : 任意の入力に対する線形時不変システムの時系列応答をシミュレーションする関数
from control.matlab import tf, ss, lsim

In [ ]:
# === ステップ応答の比較：分子の1次項の変化による影響 ===
fig, ax = plt.subplots(figsize=(3, 2.3))

# 時間ベクトルとステップ入力の定義
Td = np.arange(0, 5, 0.01)
Ud = 1 * (Td > 0.0)  # ステップ入力（t > 0で1）

# 減衰係数と自然周波数の設定
zeta = 0.4
omega_n = 3

# 線種のジェネレーター（別で定義されていると想定）
LS = linestyle_generator()

# === P1：分子が0次のみ ===
P = tf([0, omega_n**2], [1, 2*zeta*omega_n, omega_n**2])  # 分子の1次項なし
Pss = ss(P)
y, t, _ = lsim(Pss, Ud, Td)  # ステップ応答のシミュレーション
ax.plot(t, y, ls=next(LS), label='$P_1$', c='k')

# === P2：分子の1次項が正 ===
P = tf([3, omega_n**2], [1, 2*zeta*omega_n, omega_n**2])  # 分子の1次項が+3
Pss = ss(P)
y, t, _ = lsim(Pss, Ud, Td)
ax.plot(t, y, ls=next(LS), label='$P_2$', c='k')

# === P3：分子の1次項が負 ===
P = tf([-3, omega_n**2], [1, 2*zeta*omega_n, omega_n**2])  # 分子の1次項が-3
Pss = ss(P)
y, t, _ = lsim(Pss, Ud, Td)
ax.plot(t, y, ls=next(LS), label='$P_3$', c='k')

# Y軸範囲設定
ax.set_ylim(-0.5, 1.5)

# 軸ラベルや凡例などを整える（カスタム関数を使用）
plot_set(ax, 't', 'y', 'best')